In [1]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn as sk
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy import stats
from scipy.stats import ttest_ind_from_stats
from scipy.stats import ttest_ind
from statsmodels.stats.diagnostic import het_breuschpagan
import numpy as np
import pylab
import os

%matplotlib inline

In [4]:
print(os.getcwd())


df = pd.read_csv("./Burns_006157786/Burns_006157786.csv", encoding='latin-1')
#df = pd.read_csv("Burns_006157786/dataset.csv", encoding='latin-1')
df = df.loc[df['state'].isin(["Wisconsin","Illinois","Iowa"])]


#df.isnull().any()
print(df.describe())
df =df.drop(['BLOCKID'], axis=1) #Almost no records actually had values in this column, wont be using this data anyways
df = df.dropna()
df['Ratio']= df['male_pop'] / (df['male_pop'] + df['female_pop'])
print(df.describe())
#Only dropped 34 out of 2740 records, so not a super substantial loss of data. Also because these were missing houshold income
#data they werent of much use other than looking at distributions of other characteristics 
#"C:\Users\pburn\OneDrive\Documents\Burns_006157786\HiData.csv"





C:\Users\pburn


FileNotFoundError: [Errno 2] File b'.../Burns_006157786/Burns_006157786.csv' does not exist: b'.../Burns_006157786/Burns_006157786.csv'

In [ ]:
sns.scatterplot(df['married'],df['hi_median'] ,  color='magenta', alpha =.6)
plt.title("All 3 states")

g = sns.FacetGrid(df, col = "state")
g.map(sns.scatterplot, "married", "hi_median", alpha = .6);



In [ ]:
state = ['Illinois','Wisconsin','Iowa']
color = ['gray','blue','green']
col = ['pink','lightblue','lightgreen']
i=0
while(i<3):
    print(state[i])
    tempdf = df.loc[df['state'].isin([state[i]])]
    regressor = LinearRegression()  
    xdata = tempdf['married'].values.reshape(-1,1)
    ydata = tempdf['hi_median'].values.reshape(-1,1)
    regressor.fit(xdata,ydata)
    print('Intercept:',regressor.intercept_)
    print('Slope:',regressor.coef_)
    y_pred = regressor.predict(xdata)
    
    #plt.scatter(xdata, ydata,  color=col)
    sns.scatterplot(tempdf['married'], tempdf['hi_median'],color=col[i],alpha=.7)
    plt.title(state[i])
    axes = plt.gca()

    axes.set_ylim([0,200000])
    axes.set_xlim([-.05,.9])
    plt.plot(xdata, y_pred, color='red', linewidth=1)
    print('_______________________________________________________________________')
    plt.show()
    print('_______________________________________________________________________')
    sm.stats.diagnostic.het_breuschpagan(xdata,exog_het = ydata)
    res = ydata - y_pred
    bp_test = het_breuschpagan(res,xdata)
    print("BP Test Results:", bp_test)
    print('Shapiro Test Results:',stats.shapiro(xdata),"\n\n")
    i= i+1
    

i=0
while(i<3):
    plt.figure()
    tempdf = df.loc[df['state'].isin([state[i]])]
    xdata = tempdf['married'].values.reshape(-1,1)
    ydata = tempdf['hi_median'].values.reshape(-1,1)
    sns.residplot(x=xdata,y=ydata, data=tempdf, color =col[i])
    plt.xlabel("Fitted values")
    plt.title(state[i] +' Residual plot')
    i= i+1


In [ ]:
regressor = LinearRegression()  
xdata = df['married'].values.reshape(-1,1)
ydata = df['hi_median'].values.reshape(-1,1)
regressor.fit(xdata,ydata)


In [ ]:
y_pred = regressor.predict(xdata)
sns.scatterplot(df['married'], df['hi_median'],color='red',alpha=.5)
#plt.scatter(xdata, ydata,  color='lightgreen')
plt.plot(xdata, y_pred, color='black', linewidth=1.5)
plt.title("All 3 states")
plt.show()


In [ ]:
sns.residplot(x=xdata,y=ydata, data=df, color ='red')
plt.xlabel("Fitted values")
plt.title('Residual plot all 3 states')



In [ ]:
sm.stats.diagnostic.het_breuschpagan(xdata,exog_het = ydata)
res = ydata - y_pred
bp_test = het_breuschpagan(res,xdata)
print(bp_test)
#Because the P-value is less than .05, this indicates heteroscedasticity 


In [ ]:
stats.shapiro(xdata)
#not a normal data set but it doesn't have to be

In [ ]:
#qq = sm.qqplot(res, stats.t, fit=True, line='45')
#sm.qqplot(res, loc = 20, scale = 5 ,  line='45', color='green')
mod_fit = sm.OLS(ydata, xdata).fit()
res = mod_fit.resid # residuals

fig = sm.qqplot(res,line='s')
plt.title("qq plot")




In [ ]:
#The qqplot suggests that the data is skewed to the right. This is expected as we know there are areas
# with extremely wealthy people concentrated in rich neighborhoods

In [ ]:
bp_test = het_breuschpagan(res,xdata)
print("BP Test Results:", bp_test)
print('Shapiro Test Results:',stats.shapiro(xdata),"\n\n")

In [ ]:
#Significance test

#IL
tempdf = df.loc[df['state'].isin(['Illinois'])]
xdata = tempdf['married'].values.reshape(-1,1)
xdata = sm.add_constant(xdata)
ydata = tempdf['hi_median'].values.reshape(-1,1)
mod_fitIL = sm.OLS(ydata, xdata).fit()
y_predIL = mod_fitIL.predict(xdata)
resIL = mod_fitIL.resid # residuals

#IO
tempdf = df.loc[df['state'].isin(['Iowa'])]
xdata = tempdf['married'].values.reshape(-1,1)
xdata = sm.add_constant(xdata)
ydata = tempdf['hi_median'].values.reshape(-1,1)
mod_fitIO = sm.OLS(ydata, xdata).fit()
y_predIO = mod_fitIO.predict(xdata)
resIO = mod_fitIO.resid # residuals

#WI
tempdf = df.loc[df['state'].isin(['Wisconsin'])]
xdata = tempdf['married'].values.reshape(-1,1)
xdata = sm.add_constant(xdata)
ydata = tempdf['hi_median'].values.reshape(-1,1)
mod_fitWI = sm.OLS(ydata, xdata).fit()
y_predWI = mod_fitWI.predict(xdata)
resWI = mod_fitWI.resid # residuals

#notIL (complement)
tempdf = df.loc[df['state'].isin(['Wisconsin', 'Iowa'])]
xdata = tempdf['married'].values.reshape(-1,1)
xdata = sm.add_constant(xdata)
ydata = tempdf['hi_median'].values.reshape(-1,1)
mod_fitC = sm.OLS(ydata, xdata).fit()
y_predC = mod_fitC.predict(xdata)
resC = mod_fitC.resid # residuals

print(stats.kruskal(y_predIL,y_predIO,y_predWI))
#Pvalue is way less than .05, definitely a significant difference. This suggests that the difference in predicted y values
stats.kruskal(resIL,resIO,resWI)

print(mod_fitIL.summary(),mod_fitIO.summary(),mod_fitWI.summary(),mod_fitC.summary())

In [ ]:
print('IL',mod_fitIL.params)
print('IA',mod_fitIO.params)
print('WI',mod_fitWI.params)

print('ALL',mod_fitC.params)


In [ ]:
ttest_ind_from_stats(mean1=123959.8, std1=3815.72, nobs1=1571, mean2=91215.8, std2=2989.49, nobs2=1135)